## Signatures

A signature is a declarative specification of input/output behavior of a DSPy module.

## Why should I use a DSPy Signature?

Long Answer: Most people coerce LMs to do tasks by hacking long, brittle prompts. Or by collecting/generating data for fine-tuning.

Not because DSPy optimizers are more creative than humans, but simply because they can try more things and tune the metrics directly.

## Inline DSPy Signatures

In [1]:
import dspy 


/home/none/Documents/sage_in_a_desktop/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<b>Example A: Sentiment Classificatio</b>

In [3]:
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300)
dspy.configure(lm=gpt3_turbo)

In [4]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
classify(sentence=sentence).sentiment

'Positive'

<b>Example B: Summarization</b>

In [5]:
# Example from the XSum dataset.
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

The 21-year-old player made seven appearances and scored one goal for West Ham in the Europa League. He had loan spells in League One with Blackpool and Colchester United, scoring twice for the latter. He has now transferred to Barnsley, but the length of his contract has not been disclosed.


In [7]:
print("Rationale:", response.rationale)
# Rational as for CoT

Rationale: produce the summary. We need to highlight the player's performance for West Ham, his loan spells in League One, and his transfer to Barnsley.


## Class-based DSPy Signatures

<b>Example C: Classification</b>

Some optimizers in DSPy, like SignatureOptimizer, can take this simple docstring and then generate more effective variants if needed.

In [8]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""

    sentence = dspy.InputField()
    sentiment = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion


In [9]:
classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='Fear'
)

In [15]:
sentence = "I think this entire world feels vain"  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> emotion') # see one variable to other variable
classify(sentence=sentence).emotion

'Sadness'

In [20]:
question = "Why are we born?"  # example from the SST-2 dataset.

world = dspy.ChainOfThought('question -> answer') # see one variable to other variable
world(question=question).answer

'We are born to experience life, learn and grow, and contribute to the world in our own unique way.'

<b>Example D: A metric that evaluates faithfulness to citations</b>

In [16]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    faithfulness = dspy.OutputField(desc="True/False indicating if text is faithful to context")

context = "The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."

text = "Lee scored 3 goals for Colchester United."

In [17]:

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
faithfulness(context=context, text=text)

Prediction(
    rationale="produce the faithfulness. We know that Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. However, there is no mention of him scoring three goals for Colchester United.",
    faithfulness='False'
)